In [12]:
# import required packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as snn
import cv2
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import legacy as legacy_optimizers

In [13]:
# Initialize image data generator with rescaling
train_data_gen = ImageDataGenerator(rescale=1./255)
validation_data_gen = ImageDataGenerator(rescale=1./255)

In [ ]:
# Preprocess all train images
train_generator = train_data_gen.flow_from_directory(
        './data/train', 
        target_size=(48, 48),
        batch_size=128,
        color_mode="grayscale",
        class_mode='categorical')

In [ ]:
# Preprocess all test images
validation_generator = validation_data_gen.flow_from_directory(
        './data/test',
        target_size=(48, 48),
        batch_size=128,
        color_mode="grayscale",
        class_mode='categorical')

In [ ]:
# create model structure
model = Sequential()

model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48, 48, 1)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dropout(0.25))

model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(6, activation='softmax'))

cv2.ocl.setUseOpenCL(False)

model.compile(loss='categorical_crossentropy', optimizer=legacy_optimizers.Adam(lr=0.000001), metrics=['accuracy'])

In [ ]:
emotion_model_info = model.fit_generator(
        train_generator,
        steps_per_epoch=28709 // 128,
        epochs=80,
        validation_data=validation_generator,
        validation_steps=7178 // 128)

In [19]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)

# save trained model weight in .h5 file
model.save_weights('model.h5')

In [ ]:
# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(validation_generator)
print(f"Test accuracy: {test_accuracy*100:.2f}%")
print(f"Test loss: {test_loss:.2f}")

# Make predictions on the test data
predictions = model.predict(validation_generator)
predicted_classes = np.argmax(predictions, axis=1)

# Get the true labels from the generator
true_classes = validation_generator.classes
class_labels = list(validation_generator.class_indices.keys())

# Print classification report
from sklearn.metrics import classification_report
report = classification_report(true_classes, predicted_classes, target_names=class_labels)
print(report)

In [ ]:
# This will show some pictures that model predict and and actual 
test_images, test_labels = next(validation_generator)

# Get predictions for this batch
predictions = model.predict(test_images)

# Function to plot the results
def plot_image_predictions(images, labels, predictions):
    n = len(images)  # Number of images
    fig, axes = plt.subplots(n, 2, figsize=(10, 3*n))
    fig.subplots_adjust(hspace=0.9, wspace=0.4)

    for i, (img, label, prediction) in enumerate(zip(images, labels, predictions)):
        ax = axes[i, 0]
        ax.imshow(img.squeeze(), cmap='gray')
        ax.title.set_text('Actual: {}'.format(np.argmax(label)))
        ax.axis('off')

        ax = axes[i, 1]
        bars = ax.bar(range(len(prediction)), prediction, color='blue')
        ax.set_xticks(range(len(prediction)))
        ax.set_xticklabels(['Angry', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral'], rotation=90)
        ax.set_title("Predicted probabilities")
        ax.set_ylim([0, 1])
        # Highlight the prediction with the highest probability
        bars[np.argmax(prediction)].set_color('red')

    plt.show()

# Convert grayscale images to RGB for display
test_images_rgb = np.repeat(test_images, 3, axis=-1)

# Plot the image predictions for the first few test images
plot_image_predictions(test_images_rgb[:10], test_labels[:10], predictions[:10])

In [ ]:
# to plot the graph of accuracy and loss
plt.plot(emotion_model_info.history['accuracy'])
plt.plot(emotion_model_info.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
plt.savefig('accuracy.png')